In [2]:
import warnings
import os
from pyspark.sql import SparkSession, functions as F
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import plotly
import kaleido
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

#warnings.filterwarnings(action='once')
#ps.set_option('compute.ops_on_diff_frames', True)

In [4]:
file_path = '../data/raw/Loan_status_2007-2020Q3.gzip'
data_dict_path = '../data/raw/LCDataDictionary.xlsx'

In [5]:
pdf = ps.read_csv(path=file_path, header='infer', dtype='str').drop('_c0', axis=1)

23/01/15 23:00:12 WARN Utils: Your hostname, sandeep-XPS-15-7590 resolves to a loopback address: 127.0.1.1; using 192.168.86.40 instead (on interface wlp59s0)
23/01/15 23:00:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/15 23:00:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [6]:
pdf1 = pdf[pdf.annual_inc != 'MORTGAGE']
pdf2 = pdf[pdf.annual_inc == 'MORTGAGE']
pdf2a = pdf2.loc[:,:'sub_grade']
emp_t = pdf2.T.loc['emp_title']
pdf2b = pdf2.loc[:,'emp_title':].T.shift(-1).T
emp_t = emp_t + pdf2b.T.loc['emp_title']
pdf2 = pdf2a.join(pdf2b, how='left')
pdf = pdf1.append(pdf2 ,ignore_index=True)

23/01/15 23:00:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


23/01/15 23:00:38 WARN MemoryStore: Not enough space to cache rdd_12_9 in memory! (computed 30.9 MiB so far)
23/01/15 23:00:38 WARN BlockManager: Persisting block rdd_12_9 to disk instead.
23/01/15 23:00:38 WARN MemoryStore: Not enough space to cache rdd_12_7 in memory! (computed 30.9 MiB so far)
23/01/15 23:00:38 WARN BlockManager: Persisting block rdd_12_7 to disk instead.
23/01/15 23:00:38 WARN MemoryStore: Not enough space to cache rdd_12_0 in memory! (computed 30.7 MiB so far)
23/01/15 23:00:38 WARN BlockManager: Persisting block rdd_12_0 to disk instead.
23/01/15 23:00:38 WARN MemoryStore: Not enough space to cache rdd_12_6 in memory! (computed 31.3 MiB so far)
23/01/15 23:00:38 WARN BlockManager: Persisting block rdd_12_6 to disk instead.
23/01/15 23:00:38 WARN MemoryStore: Not enough space to cache rdd_12_1 in memory! (computed 31.1 MiB so far)
23/01/15 23:00:38 WARN BlockManager: Persisting block rdd_12_1 to disk instead.
23/01/15 23:00:38 WARN MemoryStore: Not enough space to

23/01/15 23:00:45 WARN MemoryStore: Not enough space to cache rdd_12_12 in memory! (computed 70.0 MiB so far)
23/01/15 23:00:45 WARN BlockManager: Persisting block rdd_12_12 to disk instead.
23/01/15 23:00:46 WARN MemoryStore: Not enough space to cache rdd_12_8 in memory! (computed 31.2 MiB so far)
23/01/15 23:00:46 WARN MemoryStore: Not enough space to cache rdd_12_6 in memory! (computed 46.9 MiB so far)
23/01/15 23:00:46 WARN MemoryStore: Not enough space to cache rdd_12_1 in memory! (computed 20.7 MiB so far)
23/01/15 23:00:46 WARN MemoryStore: Not enough space to cache rdd_12_7 in memory! (computed 70.1 MiB so far)
23/01/15 23:00:47 WARN MemoryStore: Not enough space to cache rdd_12_3 in memory! (computed 20.8 MiB so far)
23/01/15 23:00:47 WARN MemoryStore: Not enough space to cache rdd_12_0 in memory! (computed 30.7 MiB so far)
23/01/15 23:00:47 WARN MemoryStore: Not enough space to cache rdd_12_9 in memory! (computed 46.3 MiB so far)
23/01/15 23:00:47 WARN MemoryStore: Not enough

23/01/15 23:00:50 WARN MemoryStore: Not enough space to cache rdd_12_12 in memory! (computed 70.0 MiB so far)


23/01/15 23:00:51 WARN MemoryStore: Not enough space to cache rdd_12_13 in memory! (computed 70.4 MiB so far)
23/01/15 23:00:51 WARN BlockManager: Persisting block rdd_12_13 to disk instead.
23/01/15 23:00:51 WARN MemoryStore: Not enough space to cache rdd_12_13 in memory! (computed 70.4 MiB so far)
23/01/15 23:00:51 WARN MemoryStore: Not enough space to cache rdd_12_0 in memory! (computed 69.2 MiB so far)


...

23/01/15 23:01:13 WARN MemoryStore: Not enough space to cache rdd_28_1 in memory! (computed 12.7 MiB so far)
23/01/15 23:01:13 WARN MemoryStore: Not enough space to cache rdd_28_3 in memory! (computed 12.7 MiB so far)
23/01/15 23:01:13 WARN MemoryStore: Not enough space to cache rdd_28_2 in memory! (computed 12.7 MiB so far)
23/01/15 23:01:13 WARN MemoryStore: Not enough space to cache rdd_28_4 in memory! (computed 19.1 MiB so far)


23/01/15 23:01:14 WARN MemoryStore: Not enough space to cache rdd_28_9 in memory! (computed 7.7 MiB so far)
23/01/15 23:01:14 WARN MemoryStore: Not enough space to cache rdd_28_8 in memory! (computed 7.8 MiB so far)
23/01/15 23:01:14 WARN MemoryStore: Not enough space to cache rdd_28_5 in memory! (computed 7.8 MiB so far)
23/01/15 23:01:14 WARN MemoryStore: Not enough space to cache rdd_28_12 in memory! (computed 8.2 MiB so far)
23/01/15 23:01:14 WARN MemoryStore: Not enough space to cache rdd_28_7 in memory! (computed 8.3 MiB so far)
23/01/15 23:01:14 WARN MemoryStore: Not enough space to cache rdd_28_6 in memory! (computed 12.8 MiB so far)
23/01/15 23:01:14 WARN MemoryStore: Not enough space to cache rdd_28_10 in memory! (computed 5.4 MiB so far)


/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


23/01/15 23:01:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/15 23:01:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/15 23:01:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/15 23:01:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/15 23:01:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [7]:
# Density plot of interest rate
fig = pdf.int_rate.str.strip().str.strip('%').astype('float64').dropna().plot.density(bw_method=0.3)

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

...

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

In [11]:
# Density plot of term
pdf.term.str.rstrip('months').str.strip().astype('float64').dropna().plot.hist().write_image("../reports/figures/term.png", engine="kaleido")

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

...

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

In [12]:
# Density plot of FICO
pdf.fico_range_low.astype('float64').dropna().plot.hist(bins=25).write_image("../reports/figures/FICO_lb.png", engine="kaleido")

In [13]:
# Density plot of utilization
util = pdf.revol_util.str.strip().str.strip('%').astype('float64').dropna()
util.where(util <= 100.0).dropna().plot.density(bw_method=0.3).write_image("../reports/figures/utilization.png", engine="kaleido")

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

...

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/sandeep/projects/python/kaggle/lendingclub_ds/.venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inste

In [ ]:
loan_details = ['id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'issue_d',
 'url',
 'purpose',
 'application_type',
               ]

cust_demo = ['emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'title',
 'zip_code',
 'addr_state',
 'annual_inc_joint',
            ]

bureau_debt = ['dti',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'dti_joint',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_inq',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_op_past_12m',
 'percent_bc_gt_75',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'revol_bal_joint',
 'sec_app_earliest_cr_line',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_open_acc',
 'sec_app_revol_util',
 'sec_app_open_act_il',
 'sec_app_num_rev_accts',
 'sec_app_chargeoff_within_12_mths',
 'sec_app_collections_12_mths_ex_med',
              ]

bureau_delnq = ['delinq_2yrs',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_revol_delinq',
 'num_accts_ever_120_pd',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'pct_tl_nvr_dlq',
 'pub_rec_bankruptcies',
               ]

scores = ['grade',
 'sub_grade',
 'fico_range_low',
 'fico_range_high',
 'sec_app_fico_range_low',
 'sec_app_fico_range_high',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
         ]

repay_loan = ['loan_status',
 'pymnt_plan',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'acc_now_delinq',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'deferral_term',
 'hardship_amount',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_length',
 'hardship_dpd',
 'hardship_loan_status',
 'orig_projected_additional_accrued_interest',
 'hardship_payoff_balance_amount',
 'hardship_last_payment_amount',
 'debt_settlement_flag'
             ]


feature_sets = {
    'loan_details': loan_details,
    'cust_demo': cust_demo,
    'bureau_debt': bureau_debt,
    'bureau_delnq': bureau_delnq,
    'scores': scores,
    'repay_loan': repay_loan,
    }

In [23]:
col_strs = ['grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status',
           'loan_status', 'pymnt_plan', 'url', 'purpose', 'title', 'zip_code', 'addr_state', 
           'initial_list_status', 'application_type', 'verification_status_joint', 'hardship_flag', 
           'hardship_type', 'hardship_reason', 'hardship_status', 'hardship_loan_status', 'debt_settlement_flag', 
           ]

col_dts = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d',
           'sec_app_earliest_cr_line', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 
          ]

col_oth = ['int_rate', 'revol_util', 'sec_app_revol_util', 'term']

col_num = [col for col in pdf.columns if col not in col_strs+col_dts+col_oth]

In [24]:
pdf[col_num] = pdf[col_num].astype('float64')

# converting percentage strings
pdf['int_rate'] = pdf.int_rate.str.strip().str.strip('%').astype('float64')/100
pdf['revol_util'] = pdf.revol_util.str.strip().str.strip('%').astype('float64')/100
pdf['sec_app_revol_util'] = pdf.sec_app_revol_util.str.strip().str.strip('%').astype('float64')/100

# Converting term suffixes
pdf['term'] = pdf.term.str.rstrip('months').str.strip().astype('float64')